# Multiple calls

The hack that works here, is that `commit_files` takes a json string that looks like this:

```json
{
   "files:
   [
      {"path" : "path/to file.txt", "code" : "foo baz" }
  ]
}
```

It's super hacky, and depends on the model spitting out a very large an syntactically correct json.

If you're here, you really should be thinking long and hard about workflows at this point.

Additionally, this three step process is also pushing llama-3.3-70b to its limit, and about 20% of the time it stops before actually writing the code or committing (another reason to be using workflows).

Note again the `system_prompt` which is the key to making these things work. You need to carefully explain the workflow step by step, and which tool to use at each step.

In [1]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall git+https://github.com/The-AI-Alliance/gofannon.git --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 538.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.0/457.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.4 MB/s 

In [2]:
from openai import OpenAI
from google.colab import userdata
import json
# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=userdata.get('deepinfra'),
    base_url="https://api.deepinfra.com/v1/openai",
)

from gofannon.github import GetRepoContents, \
  ReadIssue, \
  CommitFiles
from gofannon.basic_math import complex_response


tool_list = [F(api_key = userdata.get('github-token')) for F in [GetRepoContents,
                                                                 ReadIssue, ]]
cf = CommitFiles(api_key=userdata.get('github-token'), git_user_email='trevor.d.grant@gmail.com', git_user_name='bottrevo')
tool_list.append(cf)

tool_map = {f.name: f.fn for f in tool_list}
tool_desc_list = [f.definition for f in tool_list]

In [3]:
# Sequential Actions need to be defined in the
system_prompt = """
To analyze code then create issues is a multi step process:
Step 1 : Get the code (use get_repo_contents). Only do this once, for the entire repo.
Step 2 : Get the issue and comments (use read_issue)
Step 3 : Fix the issue, and be very careful to pass the full and correct path in a json string to commit_files (use commit_files)
Do not terminate until all three steps have been completed.
"""

user_prompt = "In the repository 'https://github.com/The-AI-Alliance/gofannon/', make two new text files with names 'foo.txt' and 'baz.txt', fill each up with some babble, and commit them to a branch called 'commit_files_demo'"


out = complex_response(prompt=user_prompt,
                       system=system_prompt,
                       tool_desc_list=tool_desc_list,
                       openai_like_client=openai,
                       model_name= "meta-llama/Llama-3.3-70B-Instruct",
                       tool_map=tool_map,
                       max_thoughts=5,
                       temperature=0.1)

print(out['last_response'].choices[0].message.content)


thinking...
This means the two files 'foo.txt' and 'baz.txt' have been successfully created and committed to the 'commit_files_demo' branch in the 'https://github.com/The-AI-Alliance/gofannon' repository. The files contain the specified text.


And so they have, see for yourself at https://github.com/The-AI-Alliance/gofannon/tree/commit_files_demo